<img src="https://drive.google.com/uc?id=1cXtXdAcwedVDbapmz1pj_hULsQrhEcff" width="500"/>

---


# **CNNs: convolutional neural networks (part 2)**

#### **Morning contents/agenda**

1. Commonly used datasets in computer vision

2. Important CNN architectures

3. U-nets and upsampling (unpooling & transpose convolutions)

4. Transfer learning

5. CNN receptive field

#### **Learning outcomes**

1. Awareness of well-established CNN architectures

2. Understand how to upsample data

3. Understand how and why transfer learning is used

<br>

#### **Afternoon contents/agenda**

1. Inspection of CNN filters

2. Transfer learning from ImageNet to Bees and Ants

#### **Learning outcomes**

1. Become familiar with the effect that filters have (sometimes you can interpret them, sometimes they have abstracted the data too far to develop intuitions)

2. Hands-on knowledge on how to apply transfer learning


<br/>

---

<br/>

In [ ]:
!pip install pycm livelossplot
%pylab inline

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from livelossplot import PlotLosses
from pycm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.datasets
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torchsummary import summary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.6/608.6 kB 26.2 MB/s eta 0:00:00
Populating the interactive namespace from numpy and matplotlib


In [ ]:
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = False

    return True

device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

Cuda installed! Running on GPU!


## CNN Recap

A visual representation of a convolutional operation:

<br>

<center><img src="https://drive.google.com/uc?id=1Et5iK-HvMJbnvxiUrCK3h9l6y49jZiGa" width="400"/></center>

<br>

Which can be expressed mathematically as:

<br>

<center><img src="https://drive.google.com/uc?id=1ME4bFACe5hE9pSYyIvclE1lE542YbEL3" width="800"/></center>

<br>

Convolutions can be described with only 4 parameters:

- input size
- filter or kernel size
- stride
- padding

<br>

<center><img src="https://drive.google.com/uc?id=1I8s30COXw4SRdV97gy2Jn13ann7Ng8Uo" width="800"/></center>

<br>

A few simple examples to help clarify convolutions with different strides and paddings:

<img src="https://drive.google.com/uc?id=1QTvlkJm1-ZZuJSZhWK1_YHQMuVMOaW6F" width="200"/>
<img src="https://drive.google.com/uc?id=1XWKNVrCIl6M9Lfn0Gt91bDGwHeWfFR9A" width="200"/>
<img src="https://drive.google.com/uc?id=1jNUFxCrd_wAB7pxlzzEq1GOEyCz7GSwf" width="200"/>
<img src="https://drive.google.com/uc?id=1S2pgp7r6oQ_tqgnYnmDB-q3TcmgzKrs5" width="200"/>

<br>

To better illustrate how the input and output channels are defined in convolutional layers, consider the case where our input image has three channels (R,G,B) and the output only has one channel:

<p align = "center"><img src="https://drive.google.com/uc?id=1jyD_4d3HvulHQ5obeKJ57ANt6-U66din" width="800"/></p><p align = "center">
<i>3-channel input (RGB image) and 1-channel output example</i>
</p>

<br>

We can compare this to the case where the input has three channels (R,G,B) but the output has two channels

<br>

<p align = "center"><img src="https://drive.google.com/uc?id=1400AvvRTZkRH06-13XHDR19GILqBxwTP" width="800"/></p><p align = "center">
<i>3-channel input (RGB image) and 2-channel output example</i>
</p>

<br>

---

<br>

## 1. Commonly used datasets in computer vision

As we saw on the first week, the network capacity has to be adjusted in order to avoid overfitting to the data. In other words, very deep networks with large number of trainable parameters require big datasets because they have a lot of capacity to accomodate variations in the data.

So far we have seen MNIST and similarly small-sized datasets:

<br>

<p align = "center"><img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png" width="400"/></p><p align = "center">
<i>MNIST dataset: 60k training & 10k test images</i>
</p>





<br>

It is often desirable to have datasets of natural images, as they can be used for a broader range of applications than MNIST-like datasets. [CIFAR-10 and CIFAR-100](https://www.cs.toronto.edu/~kriz/cifar.html) are two datasets of natural images with 10 and 100 classes respectively:

<br>

<p align = "center"><img src="https://docs.pytorch.org/tutorials/_images/cifar10.png" width="400"/></p><p align = "center">
<i>CIFAR-10 dataset: 50k training & 10k test images</i>
</p>

<br>

<p align = "center"><img src="https://miro.medium.com/max/1400/0*fqFMfJeP6CuBTuYc.webp" width="400"/></p><p align = "center">
<i>CIFAR-100 dataset: 50k training & 10k test images</i>
</p>

<br>

But larger datasets exist as well. [ImageNet](https://www.image-net.org/) has been used in various competitions, and it contains more than 14 million images and 20k classes:

<p align = "center"><img src="https://i0.wp.com/syncedreview.com/wp-content/uploads/2020/06/Imagenet.jpg?fit=1400%2C600&ssl=1" width="800"/></p><p align = "center">
<i>ImageNet: >14M images and 20k classes </i>
</p>


[Here](https://pytorch.org/vision/stable/datasets.html) is a list of available datasets in `torchvision.datasets`



## 2. Important CNN architectures

Since their introduction in 1998 with LeNet-5, convolutional neural networks have evolved significantly and competed for the top spot in computer vision tasks.

You can find a [good overview here](https://towardsdatascience.com/illustrated-10-cnn-architectures-95d78ace614d) (the images below are from this website).

<br>

<img src="https://drive.google.com/uc?id=15lzuEPzqWAfhqzDoCxBkYdB1HdmjQt42" width="800"/>

<br>

<center><img src="https://drive.google.com/uc?id=1Qkxygn32yhPx0ZsObJdeFsTbY1RfJg_7" width="800"/></center>

<br>

<center><img src="https://drive.google.com/uc?id=13HyGvrfO1y9cgetruyXtm5livL09MB2A" width="800"/></center>

<br>

<center><img src="https://drive.google.com/uc?id=1CoVWz8e9DGHgtsKSd8oQDo6qL35gzLS6" width="800"/></center>

<br>

<center><img src="https://drive.google.com/uc?id=1ZC2aIndGpfFd4oAIMloafDS7Lqax1_Uu" width="800"/></center>

<br>

<center><img src="https://drive.google.com/uc?id=1B47JOoRDygFH0hW9DL6wDll_cU4NZ9vO" width="800"/></center>

<br>

<center><img src="https://drive.google.com/uc?id=1ZqGa7aPcPXmXPOJBluB6Xjzj2wDnk1qT" width="800"/></center>

<br>

As you can see, network sizes increase over time thanks to advances in computational power (better GPUs with more memory, etc):

<br>

<center><img src="https://drive.google.com/uc?id=1AQIN4za44eMsinsGxfV4YCfpHKVo-tqX" width="600"/></center>

<br>

But even this numbers are considered small in modern architectures. For example, **GPT-4** has 1.7 trillion parameters.


<br>

---

<br/>

## 3. U-nets and upsampling (unpooling & transpose convolutions)

What are the outputs of the CNNs we have seen so far?

<br>

<center><img src="https://drive.google.com/uc?id=14Bj-nbg5nbTN_7HT9EtMX8oJq5I5xTIk" width="800"/></center>

<br>

<br>

These were classification networks, and their outputs were vectors of probabilities (calculated using the Softmax activation):

<center><img src="https://drive.google.com/uc?id=13QroJYkGtpKX0jBhH2folt-Lj9q_MZ1w" width="800"/></center>

<br>

But CNNs have other applications. In the field of computer vision, a very common architecture is the **U-Net** which is a type of convolutional autoencoder (we will see autoencoders next week):

<p align = "center"><img src="https://drive.google.com/uc?id=1glxl06_zsq-2Off0E21VOQmG5k22R6_z" width="800"/></p><p align = "center">
<i> sources: <a href="https://arxiv.org/pdf/1505.04597.pdf">original unet</a>, <a href="https://www.kaggle.com/c/tgs-salt-identification-challenge"> seismic segmentation</a></i>
</p>

<br>

An important operation we perform to generate U-Net (and other architectures) is upscaling. The most common methods are:

- nearest neighbour
- "bed of nails"
- Max unpooling

<br>

<center><img src="https://drive.google.com/uc?id=1sVrf6toOeBLhYOD4v7L-OWTXX-ahQIiJ" width="800"/></center>

<br>

and

- Transposed convolution or up-convolution, but **not deconvolution!**



## Transposed convolutions

Transposed convolutions can be computed by following an easy recipe:

<center><img src="https://miro.medium.com/v2/resize:fit:4800/format:webp/1*54-7typHLLXhdvAhlku9SQ.png" width="900"/></center>


where we know that:
- `s`: stride
- `p`: padding
- `k`: kernel size

and we use this hyperparameters to calculate:
- `z`: how many zeros to insert in between pixels of my input
- `p'`: how much padding do I add around the image

But with the added caveat that, **as the name indicates**, we need to **transpose the kernel** before using it to convolve with the input. Transposing the kernel in this case implies flipping the kernel along each of its axes.

# Exercise:
Let's practice with an example. Let's try and calculate a simple case by hand:


<p align = "center"><img src="https://drive.google.com/uc?id=1JwdzoUrh2EC-4qj4fYrRUVUaHG9pyfp6" width="800"/></p><p align = "center">

To check if we have the right solution we can use [`conv_transpose2d`](https://pytorch.org/docs/stable/generated/torch.nn.functional.conv_transpose2d.html), which allows us to pass an input and a predefined filter, whereas the [`ConvTranspose2d`](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html#torch.nn.ConvTranspose2d) layer randomly initialises the weights of the kernel which is not what we need now.

In [ ]:
x = torch.tensor([[1, 5.], [2., 3.]])          ## why do I need to get an instance as below?
#x = torch.tensor([[[[1, 5.], [2., 3.]]]])     ## how many square brackets do I need, and what do they do?
print(x.shape)

kernel = torch.tensor([[0, 1.], [2., 3.]])
# kernel = torch.tensor([[[[0, 1.], [2., 3.]]]])

torch.nn.functional.conv_transpose2d(x, kernel, stride=1, padding=0)

Transpose convolutions, particularly with stride bigger than 1 can lead to checkerboard imprints on the outputs:

In [ ]:
%%html
<iframe src="https://distill.pub/2016/deconv-checkerboard/" width="1000" height="500"></iframe>

<br>

---

<br>

## 4. Transfer learning

What is transfer learning and why is it useful? A definition from the Deep Learning book by Goodfellow et al (2016):

>*Transfer learning and domain adaptation refer to the situation where what has been learned in one setting ... is exploited to improve generalization in another setting.*

<br>

- The most well-known CNN designs are **available** on-line and have been **successfully trained** on very large number of images (millions).

- In many applications we often work with a relatively **small number of images** (or data in general).

- The idea of transfer learning is to use an existing trained CNN model which tries to solve a problem of similar nature and **tailor the model** to our particular application.

The two main strategies are:

1. **Add one (or more) layers, or retrain the last layer(s) of a pre-trained network**: This strategy assumes that the filters of most of the network do a good job at extracting data features we can use. The last layers, then, act as a final fine-tunning to capture the specific features of our data.

2. **Retrain the whole network with small learning rates:** This strategy assumes that as a whole, the network captures data features well, and it only needs a bit of a ‘nudge’ to adapt the network parameters to our particular problem. In this case, we want to keep the underlying abstraction that the network does at different scales, but fine-tune it to our problem.

We will see examples of both in this afternoon exercise.

<br>

---

<br>

## 5. CNN receptive field

The receptive field is defined as the region of the input that affects the output, and it can be defined between adjacent or non-adjacent CNN layers.

<br>

<center><img src="https://drive.google.com/uc?id=1Z08dAQ5fvDBXX8XAnzK5yfpsUTRuRrKm" width="800"/></center>

<br>

Why are we interested in the receptive field in our network?

Because the receptive field will determine what are the hyperparameters
I need to ensure full receptive field on my inputs:

- filter size and stride
- number of convolutional layers in the network

[Here](https://www.baeldung.com/cs/cnn-receptive-field-size) you can find a more detailed explanation with a few formulas to compute receptive fields.

<br>

---

<br>

# Solution exercise transpose convolution:



<p align = "center"><img src="https://drive.google.com/uc?id=1JwdzoUrh2EC-4qj4fYrRUVUaHG9pyfp6" width="800"/></p><p align = "center">

<p align = "center"><img src="https://drive.google.com/uc?id=1K1QClrmvD781tnZxM2RZTtzP4QmsG_QW" width="800"/></p><p align = "center">

And check that we can get the same result with a normal convolution

In [ ]:
#x = torch.tensor([[1, 5.], [2., 3.]])          ## why do I need to get an instance as below?
x = torch.tensor([[[[1, 5.], [2., 3.]]]])     ## how many square brackets do I need, and what do they do?
print(x.shape)

kernel = torch.tensor([[[[0, 1.], [2., 3.]]]]) ## transpose the kernel here since we will use an 'normal' convolution
print(kernel.shape)

torch.nn.functional.conv_transpose2d(x, kernel, stride=1, padding=0)

torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 2])


tensor([[[[ 0.,  1.,  5.],
          [ 2., 15., 18.],
          [ 4., 12.,  9.]]]])